In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('PostMalone.csv')
df.head()

,Unnamed: 0,Artist,Title,Album,Year,Date,Lyric
0,0,Post Malone,​​rockstar,beerbongs & bentleys,2017.0,2017-09-15,post malone hahahahaha tank god ayy ayy post...
1,1,Post Malone,White Iverson,Stoney (Deluxe),2015.0,2015-02-04,double ot i'm a new three saucin' saucin' i'...
2,2,Post Malone,Congratulations,Stoney (Deluxe),2016.0,2016-11-04,post malone mmmmm yeah yeah mmmmm yeah hey p...
3,3,Post Malone,Psycho,beerbongs & bentleys,2018.0,2018-02-23,post malone damn my ap goin' psycho lil' mama ...
4,4,Post Malone,I Fall Apart,Stoney (Deluxe),2016.0,2016-12-09,ooh i fall apart ooh yeah mmm yeah she told ...


In [3]:
df.groupby('Album').count()

,Unnamed: 0,Artist,Title,Year,Date,Lyric
Album,,,,,,
August 26th,10,10,10,10,10,10
Hollywood’s Bleeding,16,16,16,16,16,16
NBA 2K20 Soundtrack,1,1,1,1,1,1
Stoney (Deluxe),17,17,17,17,17,17
The Withdrawal Deux,1,1,1,1,1,1
Unreleased Songs,43,43,43,0,0,43
beerbongs & bentleys,18,18,18,18,18,18


In [43]:
import requests
import base64

# Replace with your Client ID and Client Secret
client_id = "3df60c90620148448eb0745f7dd38101"
client_secret = "8a0a1b27ffa54808946fa7db0f8c88d7"

# Encode the client credentials
auth_str = f"{client_id}:{client_secret}"
auth_bytes = auth_str.encode("utf-8")
auth_base64 = base64.b64encode(auth_bytes).decode("utf-8")

# Request access token
auth_url = "https://accounts.spotify.com/api/token"
headers = {"Authorization": f"Basic {auth_base64}"}
data = {"grant_type": "client_credentials"}

#Using requests to get the temporary access token
response = requests.post(auth_url, headers=headers, data=data)
access_token = response.json().get("access_token")

print("Access Token:", access_token)

Access Token: BQB56Y5XWBUxfoOjhhPyEwa2w2iuxQ6fsUlD6hdS5MBGwUSVAHuY8_f5P7EjT1dKsv7ojxKjLDNRAklDCPjPsWfYMiH6fmOviJsisshGE9AupjAbq_w


In [21]:
search_url = "https://api.spotify.com/v1/search"
headers = {"Authorization": f"Bearer {access_token}"}
params = {"q": "The Kid LAROI", "type": "artist", "limit": 1}

response = requests.get(search_url, headers=headers, params=params)
artist_id = response.json()["artists"]["items"][0]["id"]

print("Artist ID:", artist_id)

albums_url = f"https://api.spotify.com/v1/artists/{artist_id}/albums"
headers = {"Authorization": f"Bearer {access_token}"}
params = {"include_groups": "album", "limit": 50} #add ,single for solo-released singles

response = requests.get(albums_url, headers=headers, params=params)
albums = response.json()["items"]

while response.json().get("next"):
    next_url = response.json()["next"]
    response = requests.get(next_url, headers=headers)
    albums.extend(response.json()["items"])


Artist ID: 2tIP7SsRs7vjIcLrU85W8J


In [22]:
albums_df = pd.DataFrame(albums)
albums_df.head()

,album_type,total_tracks,available_markets,external_urls,href,id,images,name,release_date,release_date_precision,type,uri,artists,album_group
0,album,28,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",{'spotify': 'https://open.spotify.com/album/30...,https://api.spotify.com/v1/albums/30H0cGghxhBJ...,30H0cGghxhBJ93v4h28XEt,[{'url': 'https://i.scdn.co/image/ab67616d0000...,THE FIRST TIME (DELUXE VERSION),2024-08-09,day,album,spotify:album:30H0cGghxhBJ93v4h28XEt,[{'external_urls': {'spotify': 'https://open.s...,album
1,album,20,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",{'spotify': 'https://open.spotify.com/album/63...,https://api.spotify.com/v1/albums/63IolVUykZCH...,63IolVUykZCHMlu2zu9jHS,[{'url': 'https://i.scdn.co/image/ab67616d0000...,THE FIRST TIME,2023-11-10,day,album,spotify:album:63IolVUykZCHMlu2zu9jHS,[{'external_urls': {'spotify': 'https://open.s...,album
2,album,35,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",{'spotify': 'https://open.spotify.com/album/57...,https://api.spotify.com/v1/albums/576UTio7xgY7...,576UTio7xgY7YUxrHMzxeI,[{'url': 'https://i.scdn.co/image/ab67616d0000...,F*CK LOVE 3+: OVER YOU,2021-07-27,day,album,spotify:album:576UTio7xgY7YUxrHMzxeI,[{'external_urls': {'spotify': 'https://open.s...,album


In [39]:
albums_df['href']

0    https://api.spotify.com/v1/albums/30H0cGghxhBJ...
1    https://api.spotify.com/v1/albums/63IolVUykZCH...
2    https://api.spotify.com/v1/albums/576UTio7xgY7...
Name: href, dtype: object

In [40]:
album_url = albums_df['href'][0]
headers = {"Authorization": f"Bearer {access_token}"}

response = requests.get(album_url, headers=headers)

if response.status_code == 200:
    print(response.json())
else:
    print("Error:", response.json())

{'album_type': 'album', 'total_tracks': 28, 'available_markets': ['AR', 'AU', 'AT', 'BE', 'BO', 'BR', 'BG', 'CA', 'CL', 'CO', 'CR', 'CY', 'CZ', 'DK', 'DO', 'DE', 'EC', 'EE', 'SV', 'FI', 'FR', 'GR', 'GT', 'HN', 'HK', 'HU', 'IS', 'IE', 'IT', 'LV', 'LT', 'LU', 'MY', 'MT', 'MX', 'NL', 'NZ', 'NI', 'NO', 'PA', 'PY', 'PE', 'PH', 'PL', 'PT', 'SG', 'SK', 'ES', 'SE', 'CH', 'TW', 'TR', 'UY', 'US', 'GB', 'AD', 'LI', 'MC', 'ID', 'JP', 'TH', 'VN', 'RO', 'IL', 'ZA', 'SA', 'AE', 'BH', 'QA', 'OM', 'KW', 'EG', 'MA', 'DZ', 'TN', 'LB', 'JO', 'PS', 'IN', 'BY', 'KZ', 'MD', 'UA', 'AL', 'BA', 'HR', 'ME', 'MK', 'RS', 'SI', 'KR', 'BD', 'PK', 'LK', 'GH', 'KE', 'NG', 'TZ', 'UG', 'AG', 'AM', 'BS', 'BB', 'BZ', 'BT', 'BW', 'BF', 'CV', 'CW', 'DM', 'FJ', 'GM', 'GE', 'GD', 'GW', 'GY', 'HT', 'JM', 'KI', 'LS', 'LR', 'MW', 'MV', 'ML', 'MH', 'FM', 'NA', 'NR', 'NE', 'PW', 'PG', 'PR', 'WS', 'SM', 'ST', 'SN', 'SC', 'SL', 'SB', 'KN', 'LC', 'VC', 'SR', 'TL', 'TO', 'TT', 'TV', 'VU', 'AZ', 'BN', 'BI', 'KH', 'CM', 'TD', 'KM', 'GQ'

In [37]:
tracks_url = 'https://api.spotify.com/v1/albums/30H0cGghxhBJ93v4h28XEt/tracks?offset=0&limit=50'
headers = {"Authorization": f"Bearer {access_token}"}

response = requests.get(album_url, headers=headers)
print(dict(response.json())['external_urls'])

{'spotify': 'https://open.spotify.com/album/30H0cGghxhBJ93v4h28XEt'}
